In [0]:
from pyspark.sql.functions import udf, col, when, lit

df_known_for_participation = spark.read.table("imdb_dev.silver.KnownForParticipation") 
df_participation = spark.read.table("imdb_dev.silver.Participation") 
df_rating = spark.read.table("imdb_dev.silver.TitleRating") 
df_episode = spark.read.table("imdb_dev.silver.TitleEpisode") 

df_participation_dim = df_participation.join(
    df_known_for_participation.withColumnRenamed("personId", "kfp_personId").withColumnRenamed("titleId", "kfp_titleId"),
    (df_participation["personId"] == col("kfp_personId")) & (df_participation["titleId"] == col("kfp_titleId")),
    "left"
)
df_participation_dim = (
    df_participation_dim.withColumn("isKnownForParticipation", when(col("kfp_personId").isNotNull(), lit(True)).otherwise(lit(False)))
    .join(df_rating, on="titleId", how="left")
    .join(df_episode, on="titleId", how="left")
    .drop("kfp_personId", "kfp_titleId", "characterName", "seasonNumber", "episodeNumber")
)

df_participation_dim = df_participation_dim.select("titleId", "personId", "categoryId", "parentTitleId", "averageRating", "numVotes", "isKnownForParticipation")
df_participation_dim.write.format("delta").mode("overwrite").insertInto("imdb_dev.gold.ParticipationFact")